Data Stream: http://207.251.86.229/nyc-links-cams/LinkSpeedQuery.txt

Documentation: http://a841-dotweb01.nyc.gov/datafeeds/TrafficSpeed/metadata_trafficspeeds.pdf

NYC Data Feed Website: http://www.nyc.gov/html/dot/html/about/datafeeds.shtml

### Indexes

0) **Id**: TRANSCOM Link ID

1) **Speed**: Average speed a vehicle traveled between end points on the link in the most recent interval

2) **TravelTime**: Time the average vehicle took to traverse the link

3) Status: Artifact - not useful

4) **DataAsOf**: Last time data was received from link

5) linkId: TRANSCOM Link ID (same as ID field)

6) **linkPoints**: Sequence of Lat/Long points, describes locations of the senosr links.  Google compatible polyline form.

7) **EncodedPolyLine**: http://code.google.com/apis/maps/documentation/polylineutility.html  Google compatible poly levels form.

8) **EncodedPolyLineLvls**: http://code.google.com/apis/maps/documentation/polylineutility.html

9) **Owner**: Owner of the Detector

10) Transcom_id: Artifact - Not Useful

11) **Borough**: NYC borough Brooklyn, Bronx, Manhattan, Queens, Staten Island

12) **linkName**: Description of the link location and end points

In [1]:
# Importing Library and setting environment path
import os
import sys
# set the path 

sparkPath = "C:/Users/Pranathi/Downloads/spark-2.0.2-bin-hadoop2.7/spark-2.0.2-bin-hadoop2.7"

os.environ['SPARK_HOME'] = sparkPath
sys.path.append(sparkPath + '/bin')
sys.path.append(sparkPath + '/python')
sys.path.append(sparkPath + '/python/pyspark')
sys.path.append(sparkPath + '/python/pyspark/lib')
sys.path.append(sparkPath + '/python/pyspark/lib/pyspark.zip')
sys.path.append(sparkPath + '/python/pyspark/lib/py4j-0.10.3-src.zip')
sys.path.append("C:/Program Files (x86)/Java/jre1.8.0_111/bin")

import pyspark
from pyspark.streaming import StreamingContext

sc = pyspark.SparkContext("local[2]", "trafficData")
ssc = StreamingContext(sc, 10)  # Seconds per micro-batch
sqlContext = pyspark.SQLContext(sc)

streamPath = 'http://207.251.86.229/nyc-links-cams/LinkSpeedQuery.txt'
streamFilePath = "file:/C:/Users/Pranathi/Desktop/MSBA/Fall/Harvesting Big Data/Project/data/"

import datetime
today = datetime.datetime.today()
currentDay = today.day
currentMonth = today.month
currentYear = today.year

import pandas as pd

#### Testing with a static RDD

In [2]:
# trafficRDD = sc.textFile(streamFilePath + 'trafficData_20161213-142503.txt').map(lambda line: line.split(','))

# trafficRDD.map(lambda line: line[11]).collect()

# Streaming

This section will do the actual streaming.  It needs to be run in parallel with the web scraping script in order to continuously update the folder with new files.  Spark streaming will not continuously re-read a file, and will only stream new files placed into a location.

In [2]:
trafficStream = ssc.textFileStream(streamFilePath).map(lambda line: line.split(','))

trafficStream.map(lambda line: (line[0], line[1], line[2], line[4],line[11], line[12])).foreachRDD(lambda rdd: rdd.toDF().registerTempTable("traffic"))
#trafficStream.pprint()
ssc.start()

In [3]:
#Setting the layout
import numpy as np 
import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go
import plotly 
from scipy import stats

#Change the credentials to your respective plotly credentials

plotly.tools.set_credentials_file(username='xyzaw', api_key='VWHsfjekwjriwe')

token_1 = '0rsfd5jszk'
stream_id1 = dict(token=token_1, maxpoints=87)

# Create trace
trace1 = go.Bar(x=[], y=[], stream=stream_id1, name='Average Speed')

data = [trace1]

# Edit the layout
layout = dict(title = 'Average Speeds across links in New York',
              xaxis = dict(title = 'Link Names'),
              yaxis = dict(title = 'Average Speed (miles/hour)'),
              )

fig = go.Figure(data=data, layout=layout)

# Send fig to Plotly, initialize streaming plot, open new tab
py.iplot(fig, filename='Average speeds-links')



High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~pranathim1/0 or inside your plot.ly account where it is named 'Average speeds-links'


Make sure the webscraping code is on run and has atleast 10 seconds time left when you run the below cell

In [ ]:
s_1 = py.Stream(stream_id=token_1)
s_1.open()
#s_2.open()

import time
import datetime
import numpy as np

while True:
    time.sleep( 3 )
    trafficDF = sqlContext.sql("select * from traffic LIMIT 87").toPandas()  # 155 instances updated each time
    trafficDF.columns = ['ID', 'Speed', 'TravelTime', 'Time', 'Borough', 'LinkName']  # Naming the columns

    # Removing records not in current year/month/day
    trafficDF['Time'] = pd.to_datetime(trafficDF['Time'])  # Converting time to datetime
    trafficDF = trafficDF[trafficDF['Time'].dt.year == currentYear]
    trafficDF = trafficDF[trafficDF['Time'].dt.month == currentMonth]
    trafficDF = trafficDF[trafficDF['Time'].dt.day == currentDay]
    trafficDF
    x = trafficDF['LinkName']
    y = trafficDF['Speed']
    s_1.write(dict(x=x, y=y))

s_1.close()